In [ ]:
! git clone https://github.com/bvanaken/clinical-outcome-prediction.git

Cloning into 'clinical-outcome-prediction'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 117 (delta 31), reused 88 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 199.93 KiB | 14.28 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
# Replace the filter_admission_text function in clinical-outcome-prediction/blob/master/tasks/mimic_utils.py with this one because the origianl has an error.
def filter_admission_text(notes_df) -> pd.DataFrame:
    """
    Filter text information by section and only keep sections that are known at admission time.
    """
    admission_sections = {
        "CHIEF_COMPLAINT": "chief complaint:",
        "PRESENT_ILLNESS": "present illness:",
        "MEDICAL_HISTORY": "medical history:",
        "MEDICATION_ADM": "medications on admission:",
        "ALLERGIES": "allergies:",
        "PHYSICAL_EXAM": "physical exam:",
        "FAMILY_HISTORY": "family history:",
        "SOCIAL_HISTORY": "social history:"
    }

    # Replace line break indicators
    notes_df['TEXT'] = notes_df['TEXT'].str.replace(r"\n", r"\\n", regex=True)

    # Extract each section using regex
    for key, section in admission_sections.items():
        regex_pattern = rf'(?i){section}(.+?)\\n\\n[^(\\|\d|\.)]+?:'
        extracted_data = notes_df.TEXT.str.extract(regex_pattern)

        # Debugging: Print sample extracted values
        print(f"Extracted {key} sample:", extracted_data.head())

        # Assign extracted data and handle missing values
        notes_df[key] = extracted_data[0].str.replace(r'\\n', ' ', regex=True).str.strip().fillna("")

        # Fix incorrect DataFrame modification
        notes_df.loc[notes_df[key].str.startswith("[]"), key] = ""

    # Debugging: Check how many rows will be removed
    empty_rows = notes_df[(notes_df.CHIEF_COMPLAINT == "") &
                          (notes_df.PRESENT_ILLNESS == "") &
                          (notes_df.MEDICAL_HISTORY == "")]
    print(f"Rows that would be removed: {empty_rows.shape[0]} out of {notes_df.shape[0]}")

    # Filter notes with missing main information
    notes_df = notes_df[(notes_df.CHIEF_COMPLAINT != "") |
                        (notes_df.PRESENT_ILLNESS != "") |
                        (notes_df.MEDICAL_HISTORY != "")]

    # Add section headers and combine into TEXT_ADMISSION
    notes_df = notes_df.assign(TEXT="CHIEF COMPLAINT: " + notes_df.CHIEF_COMPLAINT.astype(str)
                                    + '\n\n' +
                                    "PRESENT ILLNESS: " + notes_df.PRESENT_ILLNESS.astype(str)
                                    + '\n\n' +
                                    "MEDICAL HISTORY: " + notes_df.MEDICAL_HISTORY.astype(str)
                                    + '\n\n' +
                                    "MEDICATION ON ADMISSION: " + notes_df.MEDICATION_ADM.astype(str)
                                    + '\n\n' +
                                    "ALLERGIES: " + notes_df.ALLERGIES.astype(str)
                                    + '\n\n' +
                                    "PHYSICAL EXAM: " + notes_df.PHYSICAL_EXAM.astype(str)
                                    + '\n\n' +
                                    "FAMILY HISTORY: " + notes_df.FAMILY_HISTORY.astype(str)
                                    + '\n\n' +
                                    "SOCIAL HISTORY: " + notes_df.SOCIAL_HISTORY.astype(str))

    return notes_df

In [ ]:
%cd clinical-outcome-prediction

/content/clinical-outcome-prediction


In [ ]:
import os
import gzip
import shutil

# Define directory paths
mimic_dir = "mimic_dir"
save_dir = "mp_dir"

# Create directories if they don't exist
os.makedirs(mimic_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)

# Define file paths
compressed_files = ["NOTEEVENTS.csv.gz", "ADMISSIONS.csv.gz"]

for file in compressed_files:
    if os.path.exists(file):
        output_file = os.path.join(mimic_dir, file.replace(".gz", ""))
        with gzip.open(file, 'rb') as f_in:
            with open(output_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Unzipped: {file} -> {output_file}")
    else:
        print(f"File not found: {file}")

Unzipped: NOTEEVENTS.csv.gz -> mimic_dir/NOTEEVENTS.csv
Unzipped: ADMISSIONS.csv.gz -> mimic_dir/ADMISSIONS.csv


In [ ]:
! python tasks/mp/mp.py --mimic_dir mimic_dir --save_dir mp_dir --admission_only True

/content/clinical-outcome-prediction/tasks/mp/mp.py:24: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  mimic_notes = pd.read_csv(os.path.join(mimic_dir, "NOTEEVENTS.csv"))
Extracted CHIEF_COMPLAINT sample:                                                    0
0                               \nnausea, vomiting\n
1                             \ncoffee ground emesis
2                                           Dyspnea.
3  \nSevere abdominal and back pain\nUnable to ta...
4                                           \nAngina
Extracted PRESENT_ILLNESS sample:                                                    0
0  \n35F w/ poorly controlled Type 1 diabetes mel...
1  \nMr. [**Known lastname 52368**] is a 59M w He...
2    This is a 48 year old African\nAmerican fema...
3  \nMs [**Known lastname **] is a 73 year old fe...
4  \n60yo man with known coronary disease (AMI in...
Extracted MEDICAL_HISTORY sample:                                  

In [ ]:
import pandas as pd
import json
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("DATEXIS/CORe-clinical-mortality-prediction")

# Function to process and convert CSV to JSON
def convert_csv_to_json(csv_path, json_path):
    # Load CSV file
    df = pd.read_csv(csv_path)

    # Tokenize text column
    tokenized_texts = tokenizer(
        df["text"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=512
    )

    # Convert to JSON format
    json_data = []
    for idx, row in df.iterrows():
        json_data.append({
            "id": int(row["id"]),
            "tokens": {
                "input_ids": tokenized_texts["input_ids"][idx],
                "attention_mask": tokenized_texts["attention_mask"][idx],
            },
            "label": int(row["hospital_expire_flag"])
        })

    # Save as JSON file
    with open(json_path, "w") as f:
        json.dump(json_data, f, indent=4)

    print(f"Saved: {json_path}")

# Convert train, validation, and test CSVs to JSON
convert_csv_to_json("mp_dir/MP_IN_adm_train.csv", "mp_dir/train.json")
convert_csv_to_json("mp_dir/MP_IN_adm_val.csv", "mp_dir/val.json")
convert_csv_to_json("mp_dir/MP_IN_adm_test.csv", "mp_dir/test.json")

Saved: mp_dir/train.json
Saved: mp_dir/val.json
Saved: mp_dir/test.json


In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define source directory and destination in Google Drive
source_dir = "mp_dir"  # This is the directory containing the JSON files
destination_dir = "/content/drive/MyDrive/mp_dir"  # Change if you want a different location

# Copy the entire mp_dir to Google Drive
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

print(f"Saved {source_dir} to {destination_dir}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved mp_dir to /content/drive/MyDrive/mp_dir


In [2]:
import torch
from torch.utils.data import Dataset
import json

class MIMICDataset(Dataset):
    def __init__(self, json_path):
        """
        Args:
            json_path (str): Path to the JSON file containing tokenized text data.
        """
        with open(json_path, "r") as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_ids = torch.tensor(item["tokens"]["input_ids"], dtype=torch.long)
        attention_mask = torch.tensor(item["tokens"]["attention_mask"], dtype=torch.long)
        label = torch.tensor(item["label"], dtype=torch.long)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label": label
        }


In [3]:
import torch
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
)
from tqdm import tqdm  # Progress bar

def evaluate_model(model, dataloader, device):
    """
    Evaluates the model on a given dataset with a tqdm progress bar.

    Args:
        model (torch.nn.Module): The trained model.
        dataloader (torch.utils.data.DataLoader): DataLoader for evaluation.
        device (torch.device): Device to run evaluation on (CPU or GPU).

    Returns:
        dict: Dictionary containing evaluation metrics.
    """
    model.to(device)  # Move model to the specified device
    model.eval()  # Set model to evaluation mode
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=True):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # Assuming model returns logits
            probs = torch.softmax(logits, dim=1)[:, 1]  # Probability of class 1 (dead)
            preds = torch.argmax(logits, dim=1)  # Get predicted class (0 or 1)

            # Store results
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    roc_auc = roc_auc_score(all_labels, all_probs)  # ROC-AUC Score
    auc_pr = average_precision_score(all_labels, all_probs)  # AUC-PR Score

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "auc_pr": auc_pr
    }

    return metrics


In [1]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the pre-trained teacher model
teacher_model_name = "DATEXIS/CORe-clinical-mortality-prediction"
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_model_name)

# Load the pre-trained student model
student_model_name = "nlpie/tiny-biobert"
student_model = AutoModelForSequenceClassification.from_pretrained(student_model_name, num_labels=2)

KeyboardInterrupt: 

In [4]:
from torch.utils.data import DataLoader

# Move models to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the test dataset
test_dataset = MIMICDataset("drive/MyDrive/mp_dir/test.json")
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
# Evaluate the Teacher model
teacher_metrics = evaluate_model(teacher_model, test_loader, device)

# Print evaluation results
print("Evaluation Results on Test Set:")
print(teacher_metrics)

Evaluating: 100%|██████████| 77/77 [05:11<00:00,  4.05s/it]

Evaluation Results on Test Set:
{'accuracy': 0.8211158623498269, 'precision': 0.31421319796954317, 'recall': 0.6039024390243902, 'f1_score': 0.41335559265442406, 'roc_auc': 0.824777072006255, 'auc_pr': 0.3898734582342921}


In [12]:
# Evaluate the Student model
student_metrics = evaluate_model(student_model, test_loader, device)

# Print evaluation results
print("Evaluation Results on Test Set:")
print(student_metrics)

Evaluating: 100%|██████████| 77/77 [00:09<00:00,  7.82it/s]

Evaluation Results on Test Set:
{'accuracy': 0.12024027692934229, 'precision': 0.10407569141193596, 'recall': 0.9765853658536585, 'f1_score': 0.1881048576529174, 'roc_auc': 0.4992023333897088, 'auc_pr': 0.10800276205139574}


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class KnowledgeDistillationLoss(nn.Module):
    def __init__(self, class_counts):
        """
        Custom knowledge distillation loss with label imbalance adjustment.

        Args:
            class_counts (list): List of class frequencies [count_0, count_1, ..., count_n].
        """
        super(KnowledgeDistillationLoss, self).__init__()
        self.class_counts = torch.tensor(class_counts, dtype=torch.float32)
        self.max_count = torch.max(self.class_counts)
        self.lambda_weights = self.class_counts / self.max_count  # Normalize to [0,1]

    def forward(self, student_probs, teacher_probs, true_labels_one_hot):
        """
        Computes the loss.

        Args:
            student_probs (torch.Tensor): Student model's predicted probabilities (batch_size, num_classes).
            teacher_probs (torch.Tensor): Teacher model's predicted probabilities (batch_size, num_classes).
            true_labels_one_hot (torch.Tensor): True labels in one-hot encoding (batch_size, num_classes).

        Returns:
            torch.Tensor: Computed loss.
        """
        device = student_probs.device
        self.lambda_weights = self.lambda_weights.to(device)  # Ensure it's on the same device

        # Reshape lambda weights for broadcasting
        lambda_weights = self.lambda_weights.unsqueeze(0)  # Shape (1, num_classes)

        # Compute adjusted target distribution t'
        adjusted_targets = lambda_weights * teacher_probs + (1 - lambda_weights) * true_labels_one_hot

        # Compute KL divergence loss
        loss = F.kl_div(student_probs.log(), adjusted_targets, reduction="batchmean")  # KL divergence

        return loss


In [6]:
import torch
from tqdm import tqdm
import torch.nn.functional as F
import os

def train_model(student_model, teacher_model, train_dataloader, eval_dataloader, optimizer, loss_fn, device, epochs=5, save_dir="checkpoints"):
    """
    Trains a student model using knowledge distillation with a teacher model.
    Evaluates and saves the student model at the end of each epoch.

    Args:
        student_model (torch.nn.Module): The student model to train.
        teacher_model (torch.nn.Module): The teacher model (used for soft targets).
        train_dataloader (torch.utils.data.DataLoader): DataLoader for training data.
        eval_dataloader (torch.utils.data.DataLoader): DataLoader for evaluation data.
        optimizer (torch.optim.Optimizer): Optimizer for updating student model parameters.
        loss_fn (function): Custom loss function for distillation.
        device (torch.device): Device to run training on (CPU or GPU).
        epochs (int): Number of epochs to train.
        save_dir (str): Directory to save model checkpoints.

    Returns:
        torch.nn.Module: The trained student model.
    """
    student_model.to(device)
    teacher_model.to(device)
    teacher_model.eval()  # Ensure teacher model is in evaluation mode

    os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist

    for epoch in range(epochs):
        student_model.train()  # Set student model to training mode
        total_loss = 0.0

        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=True):
            optimizer.zero_grad()  # Reset gradients

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)  # Shape: (batch_size,)

            # Convert labels to one-hot encoding
            labels_one_hot = F.one_hot(labels, num_classes=2).float()

            # Forward pass for teacher model
            with torch.no_grad():  # No gradient computation for teacher
                teacher_outputs = teacher_model(input_ids, attention_mask=attention_mask)
                teacher_logits = teacher_outputs.logits  # Shape: (batch_size, 2)
                teacher_probs = torch.softmax(teacher_logits, dim=1)  # Convert logits to probabilities

            # Forward pass for student model
            student_outputs = student_model(input_ids, attention_mask=attention_mask)
            student_logits = student_outputs.logits  # Shape: (batch_size, 2)
            student_probs = torch.softmax(student_logits, dim=1)  # Convert logits to probabilities

            # Compute loss
            loss = loss_fn(student_probs, teacher_probs, labels_one_hot)
            total_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch+1}/{epochs} - Training Loss: {avg_loss:.4f}")

        # Evaluate the model at the end of the epoch
        eval_metrics = evaluate_model(student_model, eval_dataloader, device)
        print(f"Evaluation Metrics after Epoch {epoch+1}: {eval_metrics}")

        # Save checkpoint
        checkpoint_path = os.path.join(save_dir, f"student_model_epoch_{epoch+1}.pth")
        torch.save(student_model.state_dict(), checkpoint_path)
        print(f"Model saved at {checkpoint_path}")

    return student_model


In [7]:
import json

# Path to your JSON file in Google Drive
file_path = "drive/MyDrive/mp_dir/train.json"

# Initialize counters
num_zeros = 0
num_ones = 0

# Open and read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

    # Count occurrences of labels
    for obj in data:
        if obj.get("label") == 0:
            num_zeros += 1
        elif obj.get("label") == 1:
            num_ones += 1

# Store counts in a list
counts = [num_zeros, num_ones]
print(counts)  # Output: [num_zeros, num_ones]


[30420, 3534]


In [10]:
import json

# Define file paths
file_path = "drive/MyDrive/mp_dir/test.json"  # Original file
mini_test_path = "drive/MyDrive/mp_dir/mini_test.json"  # Output file

# Read the first 10 JSON objects
with open(file_path, "r") as f:
    data = json.load(f)  # Load full JSON

# Extract first 10 objects
mini_test_data = data[:10]  # Slice first 10 entries

# Save to new file
with open(mini_test_path, "w") as f:
    json.dump(mini_test_data, f, indent=4)  # Pretty print for readability

print(f"Mini test file saved at: {mini_test_path}")


Mini test file saved at: drive/MyDrive/mp_dir/mini_test.json


In [8]:
# Load the test dataset
train_dataset = MIMICDataset("drive/MyDrive/mp_dir/train.json")
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

val_dataset = MIMICDataset("drive/MyDrive/mp_dir/val.json")
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

In [9]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader

# Load the pre-trained teacher model
teacher_model_name = "DATEXIS/CORe-clinical-mortality-prediction"
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_model_name)

# Load the pre-trained student model
student_model_name = "nlpie/tiny-biobert"
student_model = AutoModelForSequenceClassification.from_pretrained(student_model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/840 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/55.6M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpie/tiny-biobert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Move model to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loss_fn = KnowledgeDistillationLoss(counts)
optimizer = torch.optim.AdamW(student_model.parameters(), lr=1e-5)

trained_student_model = train_model(student_model, teacher_model, train_loader, val_loader, optimizer, loss_fn, device, epochs=10, save_dir="drive/MyDrive/mp_dir/new_checkpoints")

Epoch 1/10: 100%|██████████| 266/266 [04:31<00:00,  1.02s/it]


Epoch 1/10 - Training Loss: 0.1578


Evaluating: 100%|██████████| 39/39 [00:04<00:00,  7.94it/s]


Evaluation Metrics after Epoch 1: {'accuracy': 0.8944580277098615, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'roc_auc': 0.5857503452036482, 'auc_pr': 0.15822256742997204}
Model saved at drive/MyDrive/mp_dir/new_checkpoints/student_model_epoch_1.pth


Epoch 2/10: 100%|██████████| 266/266 [04:30<00:00,  1.02s/it]


...

Evaluation Metrics after Epoch 9: {'accuracy': 0.8569682151589242, 'precision': 0.34563758389261745, 'recall': 0.39768339768339767, 'f1_score': 0.36983842010771995, 'roc_auc': 0.8110108970017854, 'auc_pr': 0.3626368230720169}
Model saved at drive/MyDrive/mp_dir/new_checkpoints/student_model_epoch_9.pth


Epoch 10/10: 100%|██████████| 266/266 [04:30<00:00,  1.02s/it]


Epoch 10/10 - Training Loss: 0.0299


Evaluating: 100%|██████████| 39/39 [00:04<00:00,  7.89it/s]


Evaluation Metrics after Epoch 10: {'accuracy': 0.8598207008964955, 'precision': 0.3559322033898305, 'recall': 0.40540540540540543, 'f1_score': 0.37906137184115524, 'roc_auc': 0.8114299786281564, 'auc_pr': 0.36401032515850906}
Model saved at drive/MyDrive/mp_dir/new_checkpoints/student_model_epoch_10.pth


In [11]:
new_student_metrics = evaluate_model(student_model, test_loader, device)

Evaluating: 100%|██████████| 77/77 [00:09<00:00,  7.75it/s]


In [14]:
teacher_metrics = evaluate_model(teacher_model, test_loader, device)

Evaluating: 100%|██████████| 77/77 [00:54<00:00,  1.42it/s]


In [16]:
print("Student Results:")
for metric, value in new_student_metrics.items():
    print(f"  {metric}: {value:.4f}")

print("\nTeacher Results:")
for metric, value in teacher_metrics.items():
    print(f"  {metric}: {value:.4f}")


Student Results:
  accuracy: 0.8641
  precision: 0.3723
  recall: 0.4410
  f1_score: 0.4038
  roc_auc: 0.8178
  auc_pr: 0.3758

Teacher Results:
  accuracy: 0.8211
  precision: 0.3142
  recall: 0.6039
  f1_score: 0.4134
  roc_auc: 0.8248
  auc_pr: 0.3899
